# Construction Loan Estimator

## Parameters

In [1]:
# All imports
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from dateutil.relativedelta import relativedelta
import datetime
from pandas.tseries.offsets import DateOffset
import numpy as np
from math import e

# Global options
pd.set_option('display.max_columns', None)
output_notebook()

Loading BokehJS ...

In [2]:
# Set scenario parameters

# Initial Costs
property_cost = 115000
const_cost = 300000
down_payment = 80000
built_value = 600000
const_close_cost = 0.05
purch_cost = 600000
purch_close_cost = 0.05

# Terms
const_loan_years = 30
const_mos = 12
years_to_consider = 45
purch_loan_years = 30

# Ongoing costs
maint_per_sqft = 0.75
assess_rate = 0.07
mill_levy = 95
const_init_hoa = 150
purch_initi_hoa = 150

# Rent costs
initial_rent = 2400

# Economic factors
inflation = 0.02
rent_growth_rate = 0.035
const_apprec = 0.057
const_loan_rate = 0.04
invest_rate = 0.0992
purch_apprec = 0.057
purch_loan_rate = 0.04

# Property estimates
square_footage = 1750

# Conversions
tot_const_loan_amt = property_cost + const_cost - down_payment
const_loan_rate_mon = const_loan_rate / 12
invest_rate_mon = invest_rate / 12
rent_growth_rate_monthly = rent_growth_rate / 12
const_apprec_mon = const_apprec / 12
current_month = datetime.date.today().replace(day=1)
const_loan_mos = const_loan_years * 12
months_to_consider = years_to_consider * 12
inflation_mon = inflation / 12
purch_loan_mos = purch_loan_years * 12
purch_loan_rate_mon = purch_loan_rate / 12
purch_apprec_mon = purch_apprec / 12
tot_purch_loan_amt = purch_cost - down_payment

In [3]:
# Create blank table for monthly level data
monthly_data = pd.DataFrame()

# Create Period to offset the index by one, to indicate month and create periods
monthly_data['Overall Month'] = range(1, months_to_consider + 2)
monthly_data['Overall Year'] = ((monthly_data['Overall Month'] - 1) // 12)
monthly_data['Year Start'] = np.where((((monthly_data['Overall Month'] - 1) // 12) == (monthly_data['Overall Month'] - 1) / 12) | (monthly_data['Overall Month'] == 1), 1, 0)
monthly_data['Construction Month'] = np.where(monthly_data['Overall Month'] <= const_mos, monthly_data['Overall Month'], 0)
monthly_data['Construction Loan Month'] = np.where((monthly_data['Overall Month'] > const_mos) & (monthly_data['Overall Month'] < (const_mos + const_loan_mos)), monthly_data['Overall Month'] - const_mos, 0)
monthly_data['Purchase Loan Month'] = np.where(monthly_data['Overall Month'] < purch_loan_mos, monthly_data['Overall Month'], 0)

# Calculate home value
monthly_data['Build Home Value'] = np.where(monthly_data['Overall Month'] <= const_mos, 0, built_value * (e**(const_apprec_mon * (monthly_data['Overall Month'] - const_mos))))

monthly_data['Purchase Home Value'] = purch_cost * (e**(purch_apprec_mon * (monthly_data['Overall Month'])))

# Calculate loan value
monthly_data['Construction Loan Value'] = np.where(monthly_data['Construction Loan Month'] != 0, tot_const_loan_amt * (((1 + const_loan_rate_mon) ** (const_loan_mos)) - ((1 + const_loan_rate_mon)**(monthly_data['Construction Loan Month'] - 1))) / ((1 + const_loan_rate_mon)**(const_loan_mos) - 1), tot_const_loan_amt)
monthly_data['Construction Loan Value'] = np.where(monthly_data['Overall Month'] > (const_mos + const_loan_mos), 0, monthly_data['Construction Loan Value'])

monthly_data['Purchase Loan Value'] = np.where(monthly_data['Purchase Loan Month'] != 0, tot_purch_loan_amt * (((1 + purch_loan_rate_mon) ** (purch_loan_mos)) - ((1 + purch_loan_rate_mon)**(monthly_data['Purchase Loan Month'] - 1))) / ((1 + purch_loan_rate_mon)**(purch_loan_mos) - 1), tot_purch_loan_amt)
monthly_data['Purchase Loan Value'] = np.where(monthly_data['Overall Month'] > purch_loan_mos, 0, monthly_data['Purchase Loan Value'])

# Calculate disbursement payments in a linear fashion
monthly_data['Construction Disbursements'] = np.where(monthly_data['Overall Month'] <= const_mos, tot_const_loan_amt / const_mos, 0)

# Calculate monthly payments
monthly_data['Construction Mortgage Payment'] = np.where((monthly_data['Overall Month'] > const_mos) & (monthly_data['Overall Month'] < (const_mos + const_loan_mos)), tot_const_loan_amt * (const_loan_rate_mon * ((1 + const_loan_rate_mon)**const_loan_mos)) / ((1 + const_loan_rate_mon)**const_loan_mos - 1), 0)

monthly_data['Purchase Mortgage Payment'] = np.where(monthly_data['Overall Month'] < purch_loan_mos, tot_purch_loan_amt * (purch_loan_rate_mon * ((1 + purch_loan_rate_mon)**purch_loan_mos)) / ((1 + purch_loan_rate_mon)**purch_loan_mos - 1), 0)

# Calculate interest payment
monthly_data['Construction Interest Payment'] = np.where(monthly_data['Overall Month'] <= const_mos, monthly_data['Construction Disbursements'].cumsum() * const_loan_rate_mon, monthly_data['Construction Loan Value'] * const_loan_rate_mon)
monthly_data['Construction Cumulative Interest Payments'] = monthly_data['Construction Interest Payment'].cumsum()

monthly_data['Purchase Interest Payment'] = monthly_data['Purchase Loan Value'] * purch_loan_rate_mon
monthly_data['Purchase Cumulative Interest Payments'] = monthly_data['Purchase Interest Payment'].cumsum()

# Calculate principal payments
monthly_data['Construction Principal Payment'] = np.where(monthly_data['Overall Month'] > const_mos, monthly_data['Construction Mortgage Payment'] - monthly_data['Construction Interest Payment'], 0)
monthly_data['Construction Cumulative Principal Payments'] = monthly_data['Construction Principal Payment'].cumsum()

monthly_data['Purchase Principal Payment'] = monthly_data['Purchase Mortgage Payment'] - monthly_data['Purchase Interest Payment']
monthly_data['Purchase Cumulative Principal Payments'] = monthly_data['Purchase Principal Payment'].cumsum()

# Calculate property taxes
monthly_data['Construction Property Tax Payment'] = np.where(monthly_data['Overall Month'] > const_mos, (((monthly_data['Build Home Value'] * assess_rate) * (mill_levy / 1000)) / 12), 0)
monthly_data['Construction Cumulative Property Taxes'] = monthly_data['Construction Property Tax Payment'].cumsum()

monthly_data['Purchase Property Tax Payment'] = (((monthly_data['Purchase Home Value'] * assess_rate) * (mill_levy / 1000)) / 12)
monthly_data['Purchase Cumulative Property Taxes'] = monthly_data['Purchase Property Tax Payment'].cumsum()

# Calculate maintenance costs
monthly_data['Construction Maintenance Costs'] = np.where(monthly_data['Overall Month'] > const_mos, ((square_footage * maint_per_sqft) / 12), 0)
monthly_data['Construction Cumulative Maintenance Costs'] = monthly_data['Construction Maintenance Costs'].cumsum()

monthly_data['Purchase Maintenance Costs'] = ((square_footage * maint_per_sqft) / 12)
monthly_data['Purchase Cumulative Maintenance Costs'] = monthly_data['Purchase Maintenance Costs'].cumsum()

# Calculate HOA costs
monthly_data['Construction HOA Costs'] = const_init_hoa * (e**(inflation_mon * (monthly_data['Overall Month'] - 1)))
monthly_data['Construction Cumulative HOA Costs'] = monthly_data['Construction HOA Costs'].cumsum()

monthly_data['Purchase HOA Costs'] = purch_initi_hoa * (e**(inflation_mon * (monthly_data['Overall Month'] - 1)))
monthly_data['Purchase Cumulative HOA Costs'] = monthly_data['Purchase HOA Costs'].cumsum()

# Calculate total home payments
monthly_data['Construction Total Home Payments'] = monthly_data['Construction Interest Payment'] + monthly_data['Construction Principal Payment'] + monthly_data['Construction Property Tax Payment'] + monthly_data['Construction Maintenance Costs'] + monthly_data['Construction HOA Costs']
monthly_data['Construction Cumulative Home Payments'] = monthly_data['Construction Total Home Payments'].cumsum() + down_payment + (const_close_cost * tot_const_loan_amt)

monthly_data['Purchase Total Home Payments'] = monthly_data['Purchase Interest Payment'] + monthly_data['Purchase Principal Payment'] + monthly_data['Purchase Property Tax Payment'] + monthly_data['Purchase Maintenance Costs'] + monthly_data['Purchase HOA Costs']
monthly_data['Purchase Cumulative Home Payments'] = monthly_data['Purchase Total Home Payments'].cumsum() + down_payment + (purch_close_cost * tot_purch_loan_amt)

# Calculate equity metrics
monthly_data['Construction Equity Ownership Proportion'] = 1- (monthly_data['Construction Loan Value'] / tot_const_loan_amt)
monthly_data['Construction Equity Value'] = monthly_data['Construction Equity Ownership Proportion'] * monthly_data['Build Home Value']
monthly_data['Construction Equity Gain'] = monthly_data['Construction Equity Value'] - monthly_data['Construction Cumulative Home Payments']
monthly_data['Construction Gains upon Sale'] = monthly_data['Build Home Value'] - monthly_data['Construction Cumulative Home Payments'] - monthly_data['Construction Loan Value']

monthly_data['Purchase Equity Ownership Proportion'] = 1- (monthly_data['Purchase Loan Value'] / tot_purch_loan_amt)
monthly_data['Purchase Equity Value'] = monthly_data['Purchase Equity Ownership Proportion'] * monthly_data['Purchase Home Value']
monthly_data['Purchase Equity Gain'] = monthly_data['Purchase Equity Value'] - monthly_data['Purchase Cumulative Home Payments']
monthly_data['Purchase Gains upon Sale'] = monthly_data['Purchase Home Value'] - monthly_data['Purchase Cumulative Home Payments'] - monthly_data['Purchase Loan Value']

# Calculate rent payments
monthly_data['Rent Payments'] = np.where(monthly_data['Overall Month'] == 1, (initial_rent + down_payment), initial_rent * (e**(rent_growth_rate_monthly * (monthly_data['Overall Month'] - 1))))
monthly_data['Cumulative Rent Payments'] = monthly_data['Rent Payments'].cumsum()

# Calculate rent investment value
monthly_data['Rent Investment'] = down_payment * (e**(invest_rate_mon * (monthly_data['Overall Month'] - 1)))

# Calculate when intersection points
# monthly_data['Decision'] = np.where(monthly_data['Cumulative Rent Payments'] - monthly_data['Cumulative Home Payments'] > 0, 'Buy', 'Rent')

monthly_data

,Overall Month,Overall Year,Year Start,Construction Month,Construction Loan Month,Purchase Loan Month,Build Home Value,Purchase Home Value,Construction Loan Value,Purchase Loan Value,Construction Disbursements,Construction Mortgage Payment,Purchase Mortgage Payment,Construction Interest Payment,Construction Cumulative Interest Payments,Purchase Interest Payment,Purchase Cumulative Interest Payments,Construction Principal Payment,Construction Cumulative Principal Payments,Purchase Principal Payment,Purchase Cumulative Principal Payments,Construction Property Tax Payment,Construction Cumulative Property Taxes,Purchase Property Tax Payment,Purchase Cumulative Property Taxes,Construction Maintenance Costs,Construction Cumulative Maintenance Costs,Purchase Maintenance Costs,Purchase Cumulative Maintenance Costs,Construction HOA Costs,Construction Cumulative HOA Costs,Purchase HOA Costs,Purchase Cumulative HOA Costs,Construction Total Home Payments,Construction Cumulative Home Payments,Purchase Total Home Payments,Purchase Cumulative Home Payments,Construction Equity Ownership Proportion,Construction Equity Value,Construction Equity Gain,Construction Gains upon Sale,Purchase Equity Ownership Proportion,Purchase Equity Value,Purchase Equity Gain,Purchase Gains upon Sale,Rent Payments,Cumulative Rent Payments,Rent Investment
0,1,0,1,1,0,1,0.000000e+00,6.028568e+05,335000.0,520000.000000,27916.666667,0.0,2482.559536,93.055556,93.055556,1733.333333,1733.333333,0.0,0.00000,749.226203,749.226203,0.000000,0.000000,334.083132,334.083132,0.000,0.000,109.375,109.375,150.000000,150.000000,150.000000,150.000000,243.055556,9.699306e+04,3076.017668,1.090760e+05,0.0,0.000000e+00,-9.699306e+04,-4.319931e+05,0.000000,0.000000e+00,-1.090760e+05,-2.621924e+04,82400.000000,8.240000e+04,8.000000e+04
1,2,0,0,2,0,2,0.000000e+00,6.057272e+05,335000.0,519250.773797,27916.666667,0.0,2482.559536,186.111111,279.166667,1730.835913,3464.169246,0.0,0.00000,751.723624,1500.949827,0.000000,0.000000,335.673802,669.756934,0.000,0.000,109.375,218.750,150.250208,300.250208,150.250208,300.250208,336.361320,9.732942e+04,3077.858547,1.121539e+05,0.0,0.000000e+00,-9.732942e+04,-4.323294e+05,0.001441,8.727436e+02,-1.112811e+05,-2.567749e+04,2407.010218,8.480701e+04,8.066407e+04
2,3,0,0,3,0,3,0.000000e+00,6.086112e+05,335000.0,518499.050173,27916.666667,0.0,2482.559536,279.166667,558.333333,1728.330167,5192.499413,0.0,0.00000,754.229369,2255.179196,0.000000,0.000000,337.272045,1007.028979,0.000,0.000,109.375,328.125,150.500834,450.751043,150.500834,450.751043,429.667501,9.775908e+04,3079.707416,1.152336e+05,0.0,0.000000e+00,-9.775908e+04,-4.327591e+05,0.002886,1.756721e+03,-1.134769e+05,-2.512142e+04,2414.040913,8.722105e+04,8.133366e+04
3,4,0,0,4,0,4,0.000000e+00,6.115090e+05,335000.0,517744.820804,27916.666667,0.0,2482.559536,372.222222,930.555556,1725.816069,6918.315483,0.0,0.00000,756.743467,3011.922663,0.000000,0.000000,338.877898,1345.906877,0.000,0.000,109.375,437.500,150.751878,601.502921,150.751878,601.502921,522.974100,9.828206e+04,3081.564313,1.183151e+05,0.0,0.000000e+00,-9.828206e+04,-4.332821e+05,0.004337,2.652043e+03,-1.156631e+05,-2.455098e+04,2421.092144,8.964214e+04,8.200881e+04
4,5,0,0,5,0,5,0.000000e+00,6.144206e+05,335000.0,516988.077337,27916.666667,0.0,2482.559536,465.277778,1395.833333,1723.293591,8641.609074,0.0,0.00000,759.265945,3771.188608,0.000000,0.000000,340.491397,1686.398274,0.000,0.000,109.375,546.875,151.003341,752.506262,151.003341,752.506262,616.281119,9.889834e+04,3083.429274,1.213986e+05,0.0,0.000000e+00,-9.889834e+04,-4.338983e+05,0.005792,3.558822e+03,-1.178398e+05,-2.396609e+04,2428.163970,9.207031e+04,8.268956e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,537,44,0,0,0,0,7.263954e+06,7.690028e+06,0.0,0.000000,0.000000,0.0,0.000000,0.000000,249132.532905,0.000000,375446.518739,0.

In [4]:
# Shrink set to year starts for simpler graphs
year_data = monthly_data.loc[monthly_data['Year Start'] == 1]

year_data

,Overall Month,Overall Year,Year Start,Construction Month,Construction Loan Month,Purchase Loan Month,Build Home Value,Purchase Home Value,Construction Loan Value,Purchase Loan Value,Construction Disbursements,Construction Mortgage Payment,Purchase Mortgage Payment,Construction Interest Payment,Construction Cumulative Interest Payments,Purchase Interest Payment,Purchase Cumulative Interest Payments,Construction Principal Payment,Construction Cumulative Principal Payments,Purchase Principal Payment,Purchase Cumulative Principal Payments,Construction Property Tax Payment,Construction Cumulative Property Taxes,Purchase Property Tax Payment,Purchase Cumulative Property Taxes,Construction Maintenance Costs,Construction Cumulative Maintenance Costs,Purchase Maintenance Costs,Purchase Cumulative Maintenance Costs,Construction HOA Costs,Construction Cumulative HOA Costs,Purchase HOA Costs,Purchase Cumulative HOA Costs,Construction Total Home Payments,Construction Cumulative Home Payments,Purchase Total Home Payments,Purchase Cumulative Home Payments,Construction Equity Ownership Proportion,Construction Equity Value,Construction Equity Gain,Construction Gains upon Sale,Purchase Equity Ownership Proportion,Purchase Equity Value,Purchase Equity Gain,Purchase Gains upon Sale,Rent Payments,Cumulative Rent Payments,Rent Investment
0,1,0,1,1,0,1,0.000000e+00,6.028568e+05,335000.000000,520000.000000,27916.666667,0.00000,2482.559536,93.055556,93.055556,1733.333333,1733.333333,0.000000,0.000000,749.226203,749.226203,0.000000,0.000000,334.083132,334.083132,0.000,0.000,109.375,109.375,150.000000,150.000000,150.000000,150.000000,243.055556,9.699306e+04,3076.017668,1.090760e+05,0.000000,0.000000e+00,-9.699306e+04,-4.319931e+05,0.000000,0.000000e+00,-1.090760e+05,-2.621924e+04,82400.000000,8.240000e+04,8.000000e+04
12,13,1,1,0,1,13,6.028568e+05,6.382178e+05,335000.000000,510842.610547,0.000000,1599.34124,2482.559536,1116.666667,8375.000000,1702.808702,22336.133686,482.674573,482.674573,779.750835,9937.140288,334.083132,334.083132,353.679049,4469.345025,109.375,109.375,109.375,1421.875,153.030201,1969.636124,153.030201,1969.636124,2195.829573,1.080208e+05,3098.643786,1.461341e+05,0.000000,0.000000e+00,-1.080208e+05,1.598360e+05,0.017610,1.123925e+04,-1.348949e+05,-1.875891e+04,2485.487301,1.117527e+05,8.834297e+04
24,25,2,1,0,13,25,6.382178e+05,6.756530e+05,329100.527949,501312.134919,0.000000,1599.34124,2482.559536,1097.001760,21647.957919,1671.040450,42564.604243,502.339480,6401.811531,811.519087,19499.384168,353.679049,4469.345025,374.424380,8847.164055,109.375,1421.875,109.375,2734.375,156.121616,3826.031336,156.121616,3826.031336,2218.516905,1.345170e+05,3122.480533,1.834716e+05,0.017610,1.123925e+04,-1.232778e+05,1.746003e+05,0.035938,2.428175e+04,-1.591898e+05,-9.130677e+03,2574.019635,1.421509e+05,9.755601e+04
36,37,3,1,0,25,37,6.756530e+05,7.152840e+05,322960.702303,491393.373009,0.000000,1599.34124,2482.559536,1076.535674,34679.761067,1637.977910,62403.494230,522.805565,12562.103262,844.581626,29451.208618,374.424380,8847.164055,396.386545,13481.767609,109.375,2734.375,109.375,4046.875,159.275482,5719.928218,159.275482,5719.928218,2242.416102,1.612933e+05,3147.596564,2.211033e+05,0.035938,2.428175e+04,-1.370116e+05,1.913990e+05,0.055013,3.934974e+04,-1.817535e+05,2.787345e+03,2665.705465,1.736319e+05,1.077298e+05
48,49,4,1,0,37,49,7.152840e+05,7.572396e+05,316570.730688,481070.505434,0.000000,1599.34124,2482.559536,1055.235769,47460.584424,1603.568351,81836.931534,544.105471,18973.374783,878.991185,39808.485751,396.386545,13481.767609,419.636920,18388.217590,109.375,4046.875,109.375,5359.375,162.493060,7652.084356,162.493060,7652.084356,2267.595845,1.883647e+05,3174.064516,2.590451e+05,0.055013,3.934974e+04,-1.490150e+05,2.103486e+05,0.074864,5.669029e+04,-2.023548e+05,1.712395e+04,2760.657117,2.062343e+05,1.189647e+05
60,61,5,1,0,49,61,7.572396e+05,8.016561e+05,309920.421770,470327.068308,0.000000,1599.34124,2482.559536,1033.068073,59980.2

In [5]:
# Plot payment comparison
payments = figure(title='Rent vs. Buy Cumulative Cost', plot_width=1280, plot_height=720)
payments.xaxis.axis_label = "Year"
payments.yaxis.formatter.use_scientific = False
payments.yaxis.axis_label = "Cost"
payments.line(year_data['Overall Year'], year_data['Cumulative Rent Payments'], line_color='red', legend_label='Renting')
payments.line(year_data['Overall Year'], year_data['Construction Cumulative Home Payments'], line_color='blue', legend_label='Building')
payments.line(year_data['Overall Year'], year_data['Purchase Cumulative Home Payments'], line_color='green', legend_label='Purchase')
payments.legend.location = 'top_left'

show(payments)

In [6]:
# Plot returns comparison
returns = figure(title='Rent vs. Buy Cumulative Returns', plot_width=1280, plot_height=720)
returns.xaxis.axis_label = "Year"
returns.yaxis.formatter.use_scientific = False
returns.yaxis.axis_label = "Returns"
returns.line(year_data['Overall Year'], year_data['Rent Investment'], line_color='red', legend_label='Renting')
returns.line(year_data['Overall Year'], year_data['Construction Equity Gain'], line_color='blue', legend_label='Construction Equity Gain')
returns.line(year_data['Overall Year'], year_data['Construction Gains upon Sale'], line_color='purple', legend_label='Construction Gains upon Sale')
returns.line(year_data['Overall Year'], year_data['Purchase Equity Gain'], line_color='green', legend_label='Purchase Equity Gain')
returns.line(year_data['Overall Year'], year_data['Purchase Gains upon Sale'], line_color='grey', legend_label='Purchase Gains upon Sale')
returns.legend.location = 'top_left'
# Include line below if you wish to see the value of the home in relation to the equity value
# payments.line(year_data['Overall Year'], year_data['Home Value'], line_color='yellow', legend_label='Home Value')

show(returns)

